* figure out how to best implement additional principal payments
* clean up

# Application

In [1]:
import ipywidgets as widgets
import numpy as np
import pandas as pd
from bqplot import pyplot as plt
import numpy_financial as npf
from datetime import datetime
import ipysheet
import seaborn as sns

# Setup

In [2]:
# Creating the main inputs for the application 
principal = widgets.IntText(description = "Principal", value = 100000)
period_in_years = widgets.IntSlider(min = 1, max = 60, description = "Years")
period_in_months = widgets.IntSlider(min = 1, description = "Period")
rate = widgets.FloatText(description = "Interest Rate (in percentage)", value = 5)
start_date = widgets.DatePicker(value = datetime.today())
additional_payment = widgets.IntText(description = "Additional Principal payment", value = 50)

# This function changes the max value of the period based on the amount of years inputted
def update_x_range(*args):
    period_in_months.max = 12.0 * period_in_years.value
period_in_months.observe(update_x_range, 'value')

# Prints out the breakdown of the amount paid based on the inputs

def payments_per_period(rate_in_months, per, period_in_months, principal): 
    # The interest payment for a given period 
    rate_in_months = rate_in_months / 100 / 12
    period_in_months = period_in_months * 12
    interest_payment = npf.ipmt(rate_in_months, per, period_in_months, -principal)

    # The principal payment for a given period
    principal_payment = npf.ppmt(rate_in_months, per, period_in_months, -principal)

    total_payment = interest_payment + principal_payment

    print(f'Period # {per}')
    print('Interest Payment: {:,.2f}'.format(interest_payment))
    print('Principal Payment: {:,.2f}'.format(principal_payment))
    print('Total Payment: {:,.2f}'.format(total_payment))
    print('Split: {:.2%} Interest {:,.2%} Principal'.format(interest_payment / total_payment, principal_payment / total_payment))

### Creating the amortization table

# Creating the date index
def create_table(start_date, rate_in_months, period_in_years, principal):
    period_in_months = period_in_years * 12
    rate_in_months = rate_in_months / 100 / 12
    rng = pd.date_range(start= pd.to_datetime(start_date).date(), periods= period_in_months, freq='MS')
    rng

    # Creading the data frame 

    df = pd.DataFrame(index=rng, columns=['Payment','Principal','Interest','Balance'])
    df.index.name = "Period"

    # Adding the payment is easy since it will always be the same

    df['Payment'] = npf.pmt(rate_in_months, period_in_months, principal)

    # The principal and interest change over time but since we can grab the periods using df.index, we can just use that in our formula
    df['Principal'] = npf.ppmt(rate_in_months, df.reset_index().index + 1, period_in_months, -principal)
    df['Interest'] = npf.ipmt(rate_in_months, df.reset_index().index + 1, period_in_months, -principal)

    df['Cumulative Principal'] = df['Principal'].cumsum()
    df['Balance'] = principal - df['Cumulative Principal']
    pd.set_option('display.max_rows', None)
    print(f"Total Interest Paid: ${df['Interest'].sum():,.2f}")
    display(df.round(2))
    
### Creating the amortization graph
def create_graph(start_date, rate_in_months, period_in_years, principal):
    period_in_months = period_in_years * 12
    rate_in_months = rate_in_months / 100 / 12
    rng = pd.date_range(start= start_date, periods= period_in_months, freq='MS')
    rng

    # Creading the data frame 

    df = pd.DataFrame(index=rng, columns=['Payment','Principal','Interest','Balance'])
    df.index.name = "Period"

    # Adding the payment is easy since it will always be the same

    df['Payment'] = npf.pmt(rate_in_months, period_in_months, principal)

    # The principal and interest change over time but since we can grab the periods using df.index, we can just use that in our formula
    df['Principal'] = npf.ppmt(rate_in_months, df.reset_index().index + 1, period_in_months, -principal)
    df['Interest'] = npf.ipmt(rate_in_months, df.reset_index().index + 1, period_in_months, -principal)

    df['Cumulative Principal'] = df['Principal'].cumsum()
    df['Balance'] = principal - df['Cumulative Principal']
    display(sns.lineplot(x = "Period", y = 'Balance', data = df));

    

# Inputs

In [3]:
ui = widgets.VBox([principal, rate, start_date, period_in_years,period_in_months,additional_payment])
value_output = widgets.interactive_output(payments_per_period,
                                          {'rate_in_months': rate,
                                           'per': period_in_months,
                                           'period_in_months':period_in_years,
                                           'principal':principal })
display(widgets.HBox([ui,value_output]));



# Outputs

In [4]:
out = widgets.Output(layout={'border': '1px solid black'})

view_toggle = widgets.Dropdown(
    options=['Table', 'Graph'],
    value='Table',
    description='View:',
    disabled=False,
)

def toggle(x):
    if x == 'Table':
        display(table_output)
    elif x == 'Graph':
        display(graph_output)
with out: 

    table_output = widgets.interactive_output(create_table,
                                              {'start_date': start_date,
                                               'rate_in_months': rate,
                                               'period_in_years': period_in_years,
                                               'principal': principal})
    graph_output = widgets.interactive_output(create_graph,
                                              {'start_date': start_date,
                                               'rate_in_months': rate,
                                               'period_in_years': period_in_years,
                                               'principal': principal})
    widgets.interact(toggle, x = view_toggle)
    
out

Output(layout=Layout(border='1px solid black'))